In [1]:
import os, glob, torch, numpy as np, pickle
from functools import reduce
from collections import defaultdict

In [10]:
def prefix(depth):
    return " " if depth == 0 else reduce(lambda acc, item: acc + "|--", range(depth), "--") + " "
def inspect_pickle(path, depth):
    with open(path, "rb") as f:
        obj = pickle.load(f)
        if type(obj) == dict or type(obj) == defaultdict:
            keys = sorted(list(obj.keys()))[:5]
            item_keys = obj[keys[0]].keys()
            my_prefix = prefix(depth)
            for item in item_keys:
                print(f"{my_prefix}{item}")
                for key in keys:
                    value = obj[key][item]
                    type_value = type(value)
                    auto_print = set()
                    print_val = value if type_value in [int, str] else f"{value.shape}" if hasattr(value, "shape") else "" # f"{value.shape}{value[:3]}" if hasattr(value, "shape") else ""
                    print(f"{my_prefix} {key},{type_value.__name__} > {print_val}")
        del obj           
LIMIT = 5
def dfs(path, depth=0):
    if os.path.isdir(path):
        f = os.listdir(path)
        print(prefix(depth) + os.path.basename(path) + f" ({len(f)})")
        ct_fi = 0
        for v in f:
            if dfs(os.path.join(path, v), depth + 1):
                ct_fi += 1
            if ct_fi == LIMIT:
                break
        return False
    else:
        obj = None
        size = None
        additional = None
        if path.endswith("npy"):
            obj = np.load(path)
            size = obj.shape
        elif path.endswith("pt"):
            obj = torch.load(path)
            size = obj.size() if hasattr(obj, "size") else len(obj)
            if type(obj) is str:
                additional=obj
            elif type(obj) is torch.Tensor:
                additional=str(obj[:3])
        print(prefix(depth) + os.path.basename(path) +  f" (t={type(obj).__name__})" + ("" if size is None else f" {size} ") + 
             ("" if additional is None else additional))
        if path.endswith("pkl") or os.path.basename(path).endswith("pkl"):
            inspect_pickle(path, depth)
        return True

In [9]:
path = "../../tempdata/data"
# path = "../../tempdata/data"
print(f"Inspecting: {path}")
dfs(path)

Inspecting: ../../tempdata/data
 data (4)
--|-- hsqc_ms_pairs (3)
--|--|-- train.pkl (t=NoneType)
--|--|-- HSQC
--|--|--  0,ndarray > (18, 3)
--|--|--  1,ndarray > (29, 3)
--|--|--  2,ndarray > (12, 3)
--|--|--  3,ndarray > (14, 3)
--|--|--  4,ndarray > (13, 3)
--|--|-- MS
--|--|--  0,ndarray > (150, 2)
--|--|--  1,ndarray > (150, 2)
--|--|--  2,ndarray > (150, 2)
--|--|--  3,ndarray > (150, 2)
--|--|--  4,ndarray > (150, 2)
--|--|-- FP
--|--|--  0,ndarray > (6144,)
--|--|--  1,ndarray > (6144,)
--|--|--  2,ndarray > (6144,)
--|--|--  3,ndarray > (6144,)
--|--|--  4,ndarray > (6144,)
--|--|-- SMILES
--|--|--  0,str > C=C1CCC2C(OC(=O)C2(O)COC(C)=O)C2C(=C)C(O)CC12
--|--|--  1,str > C[C@H](CCC(=O)O)[C@H]1CCC2[C@H]3[C@H](C[C@H](O)[C@@]21C)[C@@]1(C)CC[C@@H](O)C[C@@H]1[C@H](O)[C@@H]3O
--|--|--  2,str > C[C@@H]1O[C@@H](Oc2c(-c3ccc(O)c(O)c3)oc3cc(O)cc(O)c3c2=O)[C@H](O)[C@H](OC(=O)c2cc(O)c(O)c(O)c2)[C@H]1O
--|--|--  3,str > COC(=O)[C@@]12Oc3ccc(-c4ccc5c(c4O)C(O)=C4C(=O)C[C@H](C)[C@@H](O)[C@]4(C

False

In [11]:
dfs("../../tempdata/hyun_fp_data/")

  (3)
--|-- hsqc_ms_pairs (3)
--|--|-- train (5)
--|--|--|-- SMILES (9794)
--|--|--|--|-- 0.pt (t=str) 45 C=C1CCC2C(OC(=O)C2(O)COC(C)=O)C2C(=C)C(O)CC12
--|--|--|--|-- 1.pt (t=str) 100 C[C@H](CCC(=O)O)[C@H]1CCC2[C@H]3[C@H](C[C@H](O)[C@@]21C)[C@@]1(C)CC[C@@H](O)C[C@@H]1[C@H](O)[C@@H]3O
--|--|--|--|-- 2.pt (t=str) 102 C[C@@H]1O[C@@H](Oc2c(-c3ccc(O)c(O)c3)oc3cc(O)cc(O)c3c2=O)[C@H](O)[C@H](OC(=O)c2cc(O)c(O)c(O)c2)[C@H]1O
--|--|--|--|-- 3.pt (t=str) 116 COC(=O)[C@@]12Oc3ccc(-c4ccc5c(c4O)C(O)=C4C(=O)C[C@H](C)[C@@H](O)[C@]4(C(=O)OC)O5)c(O)c3C(O)=C1C(=O)C[C@H](C)[C@@H]2O
--|--|--|--|-- 4.pt (t=str) 40 CCCCCc1cc(O)c2c(c1)OC(C)(C)c1ccc(C)cc1-2
--|--|--|-- FP (9794)
--|--|--|--|-- 0.pt (t=Tensor) torch.Size([6144]) tensor([0., 0., 0.])
--|--|--|--|-- 1.pt (t=Tensor) torch.Size([6144]) tensor([0., 0., 0.])
--|--|--|--|-- 2.pt (t=Tensor) torch.Size([6144]) tensor([0., 0., 0.])
--|--|--|--|-- 3.pt (t=Tensor) torch.Size([6144]) tensor([0., 0., 0.])
--|--|--|--|-- 4.pt (t=Tensor) torch.Size([6144]) ten

False